<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/DeepLearningFiles/Emotions_classification_tensorflow_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# import the colab folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Omdena Bhutan Chapter - Leveraging AI to Combat Mental Health Problems/Tasks/Task 2/combined_data.csv")
df.head()

,text,mental_state
0,oh my gosh,anxiety
1,"trouble sleeping, confused mind, restless hear...",anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",anxiety
3,I've shifted my focus to something else but I'...,anxiety
4,"I'm restless and restless, it's been a month n...",anxiety


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123608 entries, 0 to 123607
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   text          123245 non-null  object
 1   mental_state  123606 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


In [ ]:
df["mental_state"].value_counts()

,count
mental_state,
depression,48958
suicidal,25244
anxiety,17101
normal,16351
bipolar,7865
lonely,2820
stress,2669
ptsd,1397
personality disorder,1201


In [ ]:
df.isnull().sum()

,0
text,363
mental_state,2


In [ ]:
ndf = df.dropna()

In [ ]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 123244 entries, 0 to 123607
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   text          123244 non-null  object
 1   mental_state  123244 non-null  object
dtypes: object(2)
memory usage: 2.8+ MB


In [ ]:
ndf.isnull().sum()


,0
text,0
mental_state,0


In [ ]:
# prompt: genereate the text data cleaning function for using nltk and regex
# pre-process the text data using nltk and regex

import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
# Clean the text data
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z]+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
ndf["cleaned_text"] = ndf["text"].apply(clean_text)
ndf.head()

<ipython-input-11-6e9f05be23ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf["cleaned_text"] = ndf["text"].apply(clean_text)


,text,mental_state,cleaned_text
0,oh my gosh,anxiety,oh gosh
1,"trouble sleeping, confused mind, restless hear...",anxiety,troubl sleep confus mind restless heart tune
2,"All wrong, back off dear, forward doubt. Stay ...",anxiety,wrong back dear forward doubt stay restless re...
3,I've shifted my focus to something else but I'...,anxiety,shift focu someth el still worri
4,"I'm restless and restless, it's been a month n...",anxiety,restless restless month boy mean


In [ ]:
ndf.sample(10)

,text,mental_state,cleaned_text
12954,But I have to. I honestly just want to die I d...,depression,honestli want die think strength work overnigh...
281,"I don't feel good, I'm not calm, I'm just rest...",anxiety,feel good calm restless let lot dhikr
26708,watching the fireworks tonight made me feel re...,suicidal,watch firework tonight made feel relax warm re...
24438,I do not really know how to do anything. I suc...,suicidal,realli know anyth suck everyth parent hate use...
44165,going on holiday with my friend tomorrow exite...,normal,go holiday friend tomorrow exit twitter
90793,I noticed I've given up on people I do not kno...,depression,notic given peopl know post like know anyth an...
22170,So I have decided to do something pretty reckl...,suicidal,decid someth pretti reckless someth feel need ...
84843,I hate being reminded of how unremarkable and ...,depression,hate remind unremark uniqu pretti much everyth...
91473,Numb and lost in time. Anyone remember the og ...,depression,numb lost time anyon rememb og super mario bro...
40227,in the past year i got raped by a friend i tru...,depression,past year got rape friend trust deepli lost jo...


In [ ]:
ndf.drop(columns="text", inplace=True)


<ipython-input-13-72a09542e117>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf.drop(columns="text", inplace=True)


In [ ]:
ndf["mental_state"].value_counts()

,count
mental_state,
depression,48958
suicidal,25243
anxiety,17054
normal,16343
bipolar,7765
lonely,2820
stress,2587
ptsd,1397
personality disorder,1077


## Model 1: Classification by SMOTE train data balancing

In [ ]:
# let's build keras model with the embeddings of the dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

text = ndf["cleaned_text"].values
labels = ndf["mental_state"].values

labels_encoded, labels_names = pd.factorize(labels)
labels = to_categorical(labels_encoded, num_classes=len(labels_names))

# tokenize the dataset
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

vocal_size = len(tokenizer.word_index) + 1
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")

In [ ]:
# train and test split the model and SMOTE the input data
from imblearn.keras import BalancedBatchGenerator

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# generate balanced data
training_generator = BalancedBatchGenerator(X_train, y_train, batch_size=32, random_state=42)

# define the model
model = Sequential()
model.add(Embedding(vocal_size, 128, input_length=max_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(len(labels_names), activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# fit the model to the batch generator
model.fit(training_generator, epochs=10, batch_size=32, validation_data=(X_test, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


246/246 ━━━━━━━━━━━━━━━━━━━━ 223s 870ms/step - accuracy: 0.2465 - loss: 1.9707 - val_accuracy: 0.4202 - val_loss: 1.4991
Epoch 2/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 213s 867ms/step - accuracy: 0.5581 - loss: 1.2753 - val_accuracy: 0.4999 - val_loss: 1.4145
Epoch 3/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 262s 867ms/step - accuracy: 0.7348 - loss: 0.8448 - val_accuracy: 0.5592 - val_loss: 1.2666
Epoch 4/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 262s 867ms/step - accuracy: 0.8097 - loss: 0.6366 - val_accuracy: 0.4836 - val_loss: 1.5795
Epoch 5/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 213s 868ms/step - accuracy: 0.8492 - loss: 0.5144 - val_accuracy: 0.5113 - val_loss: 1.5960
Epoch 6/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 213s 868ms/step - accuracy: 0.8812 - loss: 0.4148 - val_accuracy: 0.4954 - val_loss: 1.7274
Epoch 7/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 213s 867ms/step - accuracy: 0.9060 - loss: 0.3531 - val_accuracy: 0.5090 - val_loss: 1.7961
Epoch 8/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 213s 867ms/step - accuracy: 0.9266 - loss: 0.27

- Model is overfitting on the training dataset.

## Model 2: Keras classification using imbalance data without SMOTE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# modeling using keras tf apis
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(Embedding(vocal_size, 128, input_length=max_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(GRU(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(len(labels_names), activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# fit the model to the train dataset
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
3082/3082 ━━━━━━━━━━━━━━━━━━━━ 1396s 449ms/step - accuracy: 0.6025 - loss: 1.1289 - val_accuracy: 0.7126 - val_loss: 0.8092
Epoch 2/10
3082/3082 ━━━━━━━━━━━━━━━━━━━━ 1384s 445ms/step - accuracy: 0.7278 - loss: 0.7626 - val_accuracy: 0.7199 - val_loss: 0.7591
Epoch 3/10
3082/3082 ━━━━━━━━━━━━━━━━━━━━ 1414s 449ms/step - accuracy: 0.7605 - loss: 0.6650 - val_accuracy: 0.7225 - val_loss: 0.7476
Epoch 4/10
1862/3082 ━━━━━━━━━━━━━━━━━━━━ 8:11 403ms/step - accuracy: 0.7969 - loss: 0.5759

## Model 3: Multi-label classification using imbalance data

In [ ]:
# create the dataset with 2 labels 1) emotion and 2) sentiment
# developr the training of the tensorflow or keras model without data balance

## Model 4: Classifying text using bert-emotion using hugging face library

In [ ]:
import torch
from transformers import Trainer, TrainingArguments

ndf.head()

,mental_state,cleaned_text
0,anxiety,oh gosh
1,anxiety,troubl sleep confus mind restless heart tune
2,anxiety,wrong back dear forward doubt stay restless re...
3,anxiety,shift focu someth el still worri
4,anxiety,restless restless month boy mean


In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "boltuix/bert-emotion"
emotion_detection = pipeline("sentiment-analysis", model=model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/44.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
output = emotion_detection(ndf["cleaned_text"][13454])
output

[{'label': 'sadness', 'score': 0.9954009652137756}]

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(ndf)
dataset

Dataset({
    features: ['mental_state', 'cleaned_text', '__index_level_0__'],
    num_rows: 123244
})

In [ ]:
# load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

In [ ]:
# define tokenizer
def tokenizer_func(examples):
  return tokenizer(examples["cleaned_text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenizer_func, batched=True)

Map:   0%|          | 0/123244 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['mental_state', 'cleaned_text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 123244
})

In [ ]:
# convert the input data into pytorch format
def pytorch_formate(example):
  return {
      "input_ids": torch.tensor(example["input_ids"]),
      "attention_mask": torch.tensor(example["attention_mask"]),
      "labels": example["labels"]
  }
label_map = {name: i for i, name in enumerate(ndf["mental_state"].unique())}

tokenized_dataset = tokenized_dataset.map(lambda examples: {"labels": [label_map[label] for label in examples["mental_state"]]}, batched=True)

#tokenized_data = tokenized_dataset.map(pytorch_formate)

Map:   0%|          | 0/123244 [00:00<?, ? examples/s]

In [ ]:
# set the training config and train the model
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
     output_dir="./bert_emotion_results",
     num_train_epochs=5,
     per_device_train_batch_size=2,
     logging_dir="./bert_emotion_logs",
     logging_steps=10,
     save_steps=100,
     eval_strategy="no",
     learning_rate=3e-5,
     report_to="none"  # Disable W&B auto-logging if not needed
 )

 # 6. Initialize Trainer
trainer = Trainer(
     model=model,
     args=training_args,
     train_dataset=tokenized_dataset,
     data_collator=data_collator,
 )

 # 7. Fine-tune the model
trainer.train()

Step,Training Loss
10,1.430200
20,2.194200
30,2.007400
40,1.802500
50,1.491600
60,1.765100
70,1.391900
80,1.321700
90,1.540800
100,1.466400


KeyboardInterrupt: 